In [45]:
import yfinance as yf
import pandas as pd
import ta
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import xgboost as xgb
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

In [10]:
train_data = yf.download('AMZN', start='2021-01-01', end='2023-12-31')
test_data = yf.download('AMZN', start='2024-01-01', end='2024-05-06')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [11]:
# Obliczanie wskaźników technicznych dla danych treningowych
train_data['SMA'] = ta.trend.sma_indicator(train_data['Close'], window=14)
train_data['RSI'] = ta.momentum.rsi(train_data['Close'], window=14)
train_data['MACD'] = ta.trend.macd(train_data['Close'])
train_data = train_data.dropna()

# Obliczanie wskaźników technicznych dla danych testowych
test_data['SMA'] = ta.trend.sma_indicator(test_data['Close'], window=14)
test_data['RSI'] = ta.momentum.rsi(test_data['Close'], window=14)
test_data['MACD'] = ta.trend.macd(test_data['Close'])
test_data = test_data.dropna()

In [41]:
# Przygotowanie danych do modelu
X_train = train_data[['SMA', 'RSI', 'MACD']]
y_train = (train_data['Close'].shift(-1) > train_data['Close']).astype(int)  # Sygnał kupna/sprzedaży
y_train = y_train.iloc[:-1]
X_train = X_train.iloc[:-1]

X_test = test_data[['SMA', 'RSI', 'MACD']]
y_test = (test_data['Close'].shift(-1) > test_data['Close']).astype(int)
y_test = y_test.iloc[:-1]
X_test = X_test.iloc[:-1]

In [43]:
# Przygotowanie modelu XGBoost
model = xgb.XGBClassifier()

# Definicja siatki parametrów
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 6, 9, 12],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Optymalizacja hiperparametrów
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Najlepsze parametry
best_params = grid_search.best_params_
print("Najlepsze parametry: ", best_params)

# Stworzenie modelu z najlepszymi parametrami
model = xgb.XGBClassifier(**best_params)
model.fit(X_train, y_train)

Najlepsze parametry:  {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'subsample': 1.0}


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [51]:
class MLStrategy(Strategy):
    def init(self):
        self.sma = self.I(lambda x: pd.Series(x).rolling(window=14).mean(), self.data.Close)
        self.rsi = self.I(lambda x: 100 - (100 / (1 + pd.Series(x).diff().rolling(window=14).apply(lambda y: (y[y > 0].sum() / abs(y[y < 0].sum())), raw=True))), self.data.Close)
        self.macd = self.I(lambda x: pd.Series(x).ewm(span=12, adjust=False).mean() - pd.Series(x).ewm(span=26, adjust=False).mean(), self.data.Close)

    def next(self):
        features = [self.sma[-1], self.rsi[-1], self.macd[-1]]
        signal = model.predict([features])[0]
        
        if signal == 1:
            self.buy()
        else:
            self.sell()

# Przeprowadzenie backtestu na danych testowych
bt = Backtest(test_data, MLStrategy, cash=10000, commission=.002)
stats = bt.run()
bt.plot()


c:\Users\dorot\AppData\Local\Programs\Python\Python39\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\dorot\AppData\Local\Programs\Python\Python39\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\dorot\AppData\Local\Programs\Python\Python39\lib\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
c:\Users\dorot\AppData\Local\Programs\Python\Python39\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gr

GridPlot(id='p1843', ...)

In [49]:
print(stats)

Start                     2024-02-07 00:00:00
End                       2024-05-03 00:00:00
Duration                     86 days 00:00:00
Exposure Time [%]                   73.770492
Equity Final [$]                   9386.95166
Equity Peak [$]                  10253.420649
Return [%]                          -6.130483
Buy & Hold Return [%]                9.194868
Return (Ann.) [%]                  -22.999244
Volatility (Ann.) [%]               17.521594
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -9.833887
Avg. Drawdown [%]                   -4.248113
Max. Drawdown Duration       53 days 00:00:00
Avg. Drawdown Duration       21 days 00:00:00
# Trades                                    4
Win Rate [%]                              0.0
Best Trade [%]                      -0.302439
Worst Trade [%]                      -7.48938
Avg. Trade [%]                    